In [8]:
import numpy as np
import os
import sys
sys.path.append(r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project')

import matplotlib.pyplot as plt # for plotting
%matplotlib widget
import pandas as pd # for data manipulation
from Models.load_data import *
from Models.helpers import *

%load_ext autoreload
%autoreload 2

import torch

path = r'C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Training_data'
features_list = os.listdir(os.path.join(path, r'features'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# This cell takes 19s
# Instantiate the dataset without transformations
dataset = IndependentCSVDataset(path, features_list)

# Split dataset into training and testing sets
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

feature_min, feature_max = compute_global_minmax(train_dataset)
print("Global feature min (train set only):", feature_min)
print("Global feature max (train set only):", feature_max)

global_transform = GlobalMinMaxNormalize(feature_min, feature_max)
train_dataset.dataset.transform = global_transform
test_dataset.dataset.transform = global_transform  # Same transform to prevent data leakage

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=0)



Skipping JETno91071.csv: sequence length 6001 is unexpected.
Skipping JETno97612.csv: sequence length 6001 is unexpected.
Skipping JETno99471.csv: sequence length 6001 is unexpected.
Skipping JETno87125.csv: sequence length 6001 is unexpected.
Global feature min (train set only): [ 0.0000000e+00 -3.5154782e+06 -2.6520195e+14  0.0000000e+00
 -1.4299840e+14  0.0000000e+00]
Global feature max (train set only): [7.9738760e+00 7.2445923e+03 6.6544650e+17 8.5139590e+17 1.8753850e+18
 1.6388709e+04]


In [3]:
# Create an iterator from the DataLoader
train_iterator = iter(train_loader)

# Get a single batch
batch = next(train_iterator)

# Unpack the batch (assuming your dataset returns (inputs, targets))
inputs, targets = batch

print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)


Inputs shape: torch.Size([32, 6000, 6])
Targets shape: torch.Size([32, 6000])


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss(pos_weight=compute_class_weights(train_loader))

Class counts: [1393199.0, 22801.0]
Class ratio: 61.1025390625


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:225: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(class_counts[0] / class_counts[1], dtype=torch.float32)


In [5]:
# Define the model and print the architecture, example with dummy input shapes

input_size = 6       # Number of features per time step
hidden_size = 64     # Number of LSTM hidden units
num_layers = 2       # Number of LSTM layers
output_size = 1      # Binary classification
dropout = 0.2

# Instantiate the model
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)

# Print model architecture
print(model)

# Example dummy input: batch_size=32, sequence_length=6000, input_size=6
dummy_input = inputs
dummy_output = model(dummy_input)

print("Input shape:", dummy_input.shape)  # Expected: (32, 6000, 6) -> 6 features per timestep
print("Output shape:", dummy_output.shape)  # Expected: (32, 6000,) -> one binary value per timestep
print("Targets shape:", targets.shape)

loss = criterion(dummy_output, targets)

print(loss.item())

LSTMModel(
  (lstm): LSTM(6, 64, num_layers=2, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Input shape: torch.Size([32, 6000, 6])
Output shape: torch.Size([32, 6000])
Targets shape: torch.Size([32, 6000])
1.8287842273712158


In [6]:
# With 10 rounds of optimisation, this takes roughly 30 min on my laptop
import torch
import torch.optim as optim

# Define loss and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss(pos_weight=compute_class_weights(train_loader))
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Assume `train_loader` is a DataLoader yielding batches of (input, target)
num_epochs = 2

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs, targets = inputs.to(device), targets.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        print(f'Accumulated epoch loss: {epoch_loss-1:.4f}', end='\r')
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")


Class counts: [1393199.0, 22801.0]
Class ratio: 61.1025390625


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)


Epoch [1/2], Loss: 1.39590.1670
Epoch [2/2], Loss: 1.37870.0293


In [36]:
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score

def evaluate_test_metrics(model, test_loader, criterion, device, threshold=0.5):
    """
    Evaluates the test loss, F1-score, and ROC-AUC of a trained model.

    Parameters:
    - model: The trained PyTorch model.
    - test_loader: DataLoader for the test dataset.
    - criterion: Loss function (e.g., nn.BCEWithLogitsLoss).
    - device: The device ('cuda' or 'cpu').
    - threshold: Decision threshold for binary classification (default=0.5).

    Returns:
    - Average test loss
    - F1-score
    - ROC-AUC score
    """
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    total_samples = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # Move data to device

            # Forward pass
            outputs = model(inputs)  # (batch_size, 6000) -> raw logits

            # Compute loss
            loss = criterion(outputs, targets)  # BCEWithLogitsLoss expects raw logits
            test_loss += loss.item() * inputs.size(0)
            total_samples += inputs.size(0)

            # Convert logits to probabilities using Sigmoid
            probs = torch.sigmoid(outputs)  # (batch_size, 6000) -> probabilities in [0,1]

            # Apply threshold to get binary predictions
            preds = (probs > threshold).int()  # Convert to 0 or 1

            # Flatten for metric calculations
            all_targets.extend(targets.cpu().numpy().flatten())  # Convert to 1D list
            all_predictions.extend(preds.cpu().numpy().flatten())  # Convert to 1D list

    # Compute average loss
    avg_test_loss = test_loss / total_samples

    # Compute F1-score
    f1 = f1_score(all_targets, all_predictions, average='macro')

    return avg_test_loss, f1

# Example Usage:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_loss, test_f1 = evaluate_test_metrics(model, train_loader, criterion, device)

print(f"Test Loss: {test_loss:.4f}")
print(f"F1 Score: {test_f1:.4f}")


C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)


Test Loss: 1.4318
F1 Score: 0.0145


In [15]:
# Define model parameters
input_size = 6       # Number of input features per time step
hidden_size = 32     # Transformer hidden size
num_layers = 2       # Number of Transformer encoder layers
output_size = 1      # Binary classification
dropout = 0.2
num_heads = 4        # Number of attention heads

# Instantiate the Transformer model
model = TransformerModel(input_size, hidden_size, num_layers, output_size, dropout, num_heads)

# Print model architecture
print(model)

# Example input: (batch_size=2, sequence_length=6000, input_size=6)
x_sample = torch.randn(2, 6000, 6)

# Predict with threshold 0.5
predictions, probabilities = model.predict(x_sample, threshold=0.5)

print("Predictions:\n", predictions)
print("Probabilities:\n", probabilities)


TransformerModel(
  (input_projection): Linear(in_features=6, out_features=32, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=128, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=128, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
    )
  )
  (fc1): Linear(in_features=32, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Predictions:
 tensor([[1, 1, 1,  ..., 1, 1, 1],
       

In [16]:
# With 10 rounds of optimisation, this takes roughly 30 min on my laptop
import torch
import torch.optim as optim

# Define loss and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.BCEWithLogitsLoss(pos_weight=compute_class_weights(train_loader))
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Assume `train_loader` is a DataLoader yielding batches of (input, target)
num_epochs = 2

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):

        inputs, targets = inputs.to(device), targets.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)

        loss = criterion(outputs, targets)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        print(f'Accumulated epoch loss: {epoch_loss-1:.4f}', end='\r')
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")


Class counts: [1397275.0, 18725.0]
Class ratio: 74.6208267211914
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Max Tost\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Max Tost\AppData\Local\Temp\ipykernel_13044\1614366310.py", line 24, in <module>
    outputs = model(inputs)
              ^^^^^^^^^^^^^
  File "C:\Users\Max Tost\anaconda3\Lib\site-packages\torch\nn\modules\module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Max Tost\anaconda3\Lib\site-packages\torch\nn\modules\module.py", line 1747, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Max Tost\Desktop\Notebooks\SPC Neural Network Project\Models\helpers.py", line 282, in forward
    x = self.transformer_encoder(x)  # Shape: (batch, seq_len, hidden_size)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Max Tost\a